<a href="https://colab.research.google.com/github/MichaelKru92/Projekt-ML-Modelierung/blob/main/Schritt_5_Datentransformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Datentransformation**

**Fehlende Eingaben:**

Der Umgang mit Null-Werten in den Rohdaten wurde im Hinblick auf die Fragestellung zur Vorhersage der Gesundheitskosten für die betroffenen Parameter geprüft mit folgenden Fazit:

*   STOP: Nullwerte Tabellen übergreifend beibehalten
*   REASONCODE, REASONDESCRIPTION: Entfernen der betroffenen Zeilen in Careplans-Tabelle
*   DRIVERS, PASSPORT, PREFIX, SUFFIX, MAIDEN, MARITAL: Entfernen der betroffenen Zeilen in Patients-Tabelle
*   REASONCODE, REASONDESCRIPTION: Entfernen der Spalten der Encounters-Tabelle
*   REASONCODE, REASONDESCRIPTION: Entfernen der Spalten der Medications-Tabelle
*   ENCOUNTER: Entfernen der Zeile in der Observations-Tabelle

**Zeitangaben:**

Datumsangaben (Spalten: START, STOP) werden vereinheitlicht.

In [ ]:
#CSV Daten in DataFrame speichern:
df_careplans = pd.read_csv(os.path.join(CSV_PATH, "careplans.csv"))
df_conditions = pd.read_csv(os.path.join(CSV_PATH, "conditions.csv"))
df_devices = pd.read_csv(os.path.join(CSV_PATH, "devices.csv"))
df_disease = pd.read_csv(os.path.join(CSV_PATH, "disease.csv"))
df_imaging_studies = pd.read_csv(os.path.join(CSV_PATH, "imaging_studies.csv"))
df_encounters = pd.read_csv(os.path.join(CSV_PATH, "encounters.csv"))
df_patients = pd.read_csv(os.path.join(CSV_PATH, "patients.csv"))
df_medications = pd.read_csv(os.path.join(CSV_PATH, "medications.csv"))
df_observations = pd.read_csv(os.path.join(CSV_PATH, "observations.csv"))
df_immunizations = pd.read_csv(os.path.join(CSV_PATH, "immunizations.csv"))
df_procedures = pd.read_csv(os.path.join(CSV_PATH, "procedures.csv"))
df_organizations = pd.read_csv(os.path.join(CSV_PATH, "organizations.csv"))
df_payer_transitions = pd.read_csv(os.path.join(CSV_PATH, "payer_transitions.csv"))
df_payers = pd.read_csv(os.path.join(CSV_PATH, "payers.csv"))
df_providers = pd.read_csv(os.path.join(CSV_PATH, "providers.csv"))
df_supplies = pd.read_csv(os.path.join(CSV_PATH, "supplies.csv"))

#NULL-Werte aus careplans entfernen:
df_careplans = df_careplans.dropna(subset = ["REASONCODE", "REASONDESCRIPTION"])
print(df_careplans.isnull().sum())

#Spalten ohne Relevanz für Kostenprediktion mit NULL-Werten aus patients entfernen:
df_patients = df_patients.drop(columns = ["DRIVERS", "PASSPORT", "PREFIX", "SUFFIX", "MAIDEN", "MARITAL"])
print(df_patients.isnull().sum())
print(len(df_patients))

#Spalten ohne Relevanz für Kostenprediktion und hoher Zahl an NULL-Werten aus encounters entfernen:
df_encounters = df_encounters.drop(columns = ["REASONCODE", "REASONDESCRIPTION"])
print(df_encounters.isnull().sum())
print(len(df_encounters))

#Spalten ohne Relevanz für Kostenprediktion und hoher Zahl an NULL-Werten aus medications entfernen:
df_medications = df_medications.drop(columns = ["REASONCODE", "REASONDESCRIPTION"])
print(df_medications.isnull().sum())
print(len(df_medications))

#Spalten ohne Relevanz für Kostenprediktion und hoher Zahl an NULL-Werten aus procedures entfernen:
df_procedures = df_procedures.drop(columns = ["REASONCODE", "REASONDESCRIPTION"])
print(df_procedures.isnull().sum())
print(len(df_procedures))

#NULL-Werte aus observations entfernen:
df_observations = df_observations.dropna(subset = "ENCOUNTER")
print(df_observations.isnull().sum())
print(len(df_observations))

#Um die Datumsangabe in einen String umzuformen und zu formatieren wird folgende Funktion verwendet:

COLUMN_LIST = ["START", "STOP"]
DF_DICT = {"df_careplans" : df_careplans, "df_conditions" : df_conditions,
           "df_devices" : df_devices, "df_disease" : df_disease,
           "df_encounters" : df_encounters, "df_imaging_studies" : df_imaging_studies,
           "df_immunizations" : df_immunizations, "df_medications" : df_medications,
           "df_observations" : df_observations, "df_organizations" : df_organizations,
           "df_patients" : df_patients, "df_payer_transitions" : df_payer_transitions,
           "df_payers" : df_payers, "df_procedures" : df_procedures,
           "df_providers" : df_providers, "df_supplies" : df_supplies}

def change_dateformat(df_dict=DF_DICT, cols_list=COLUMN_LIST):
    for df_name, df in df_dict.items():
        for cols in cols_list:
            if cols in df.columns:
                df[cols] = pd.to_datetime(df[cols], errors="coerce")
                df[cols] = df[cols].dt.strftime("%m/%d/%Y, %H:%M:%S")
                print(f"Datum in Spalte {cols} in DataFrame {df_name} erfolgreich formatiert.")
                print (df.head())
            else:
                print(f"Spalte {cols} ist in DataFrame {df_name} nicht vorhanden.")
    return df_dict

change_dateformat()

**Aggregation und Kreation sinnvoller Feature**

Initial war anzunehmen, dass folgende Parameter

*   Ausmaß der Medikation/Fall
*   Anzahl der durchgeführten Impfungen/Fall
*   Anzahl der Diagnosen/Fall
*   Anzahl der Proceduren/Fall
*   Alter
*   BMI
*   Geschlecht
*   Medikationskosten/Fall (Null-Werte durch Median ersetzt)

mit den Gesamtkosten eines Falles korrelieren und gegebenenfalls als Feature für eine Prediktion zu nutzen sind. Die Feature wurden extrahiert und einem DataFrame auf Basis des Encounter DataFRames hinzugefügt (df_encounters_final).

Dieser DataFrame stellt die Faktentabelle in unserer anschließend erstellten SQL-Datenbank dar und kann für Abfragen genutzt werden.

In [ ]:
AGG_DIF_DICT = {"df_medications" : df_medications, "df_procedures" : df_procedures, "df_patients" : df_patients,
               "df_immunizations" : df_immunizations, "df_observations" : df_observations, "df_conditions" : df_conditions}

#Funktion zur Berechnung der Anzahl pro Fall für eine definierte Spalte und Vorbereitung für den Merge mit df_encounters_final
def get_counts(df, group_col, count_col, new_col_name):
    agg_df_counts = df.groupby(group_col)[count_col].count().reset_index()
    agg_df_counts.rename(columns={group_col: "Id", count_col: new_col_name}, inplace=True)
    return agg_df_counts

med_counts = get_counts(AGG_DIF_DICT["df_medications"], "ENCOUNTER", "DESCRIPTION", "MEDICATION_COUNT")
vacc_counts = get_counts(AGG_DIF_DICT["df_immunizations"], "ENCOUNTER", "DESCRIPTION", "VACCINATION_COUNT")
cond_counts = get_counts(AGG_DIF_DICT["df_conditions"], "ENCOUNTER", "DESCRIPTION", "CONDITION_COUNT")
procedures_counts = get_counts(AGG_DIF_DICT["df_procedures"], "ENCOUNTER", "DESCRIPTION", "PROCEDURES_COUNT")

df_encounters_final = df_encounters.copy()
df_encounters_final = pd.merge(df_encounters_final, med_counts, on='Id', how='left')
df_encounters_final = pd.merge(df_encounters_final, vacc_counts, on='Id', how='left')
df_encounters_final = pd.merge(df_encounters_final, cond_counts, on='Id', how='left')
df_encounters_final = pd.merge(df_encounters_final, procedures_counts, on='Id', how='left')

df_encounters_final.fillna(0, inplace=True)
df_encounters_final[["MEDICATION_COUNT", "VACCINATION_COUNT", "CONDITION_COUNT", "PROCEDURES_COUNT"]] = df_encounters_final[["MEDICATION_COUNT", "VACCINATION_COUNT", "CONDITION_COUNT", "PROCEDURES_COUNT"]].astype(int)

# BMI aus observations extrahieren und Merge mit df_encounters_final
bmi_observations = df_observations[df_observations["DESCRIPTION"] == "Body Mass Index"].copy()
bmi_observations.rename(columns={"ENCOUNTER" : "Id", "VALUE" : "BMI"}, inplace=True)
df_encounters_final = pd.merge(df_encounters_final, bmi_observations[["Id","BMI"]], on="Id", how= "left") #Nur Spalte BMI im Merge

# Geschlecht aus patients extrahieren und Merge mit df_encounters_final
gender_patients = df_patients[["Id","GENDER"]].copy()
gender_patients.rename(columns ={"Id" : "PATIENT"}, inplace=True)
df_encounters_final = pd.merge(df_encounters_final, gender_patients[["PATIENT","GENDER"]], on="PATIENT", how= "left")

#Medikationskosten pro Fall Null-Werte durch Median ersetzt
med_cost = df_medications.groupby("ENCOUNTER")["TOTALCOST"].sum().reset_index()
med_cost.rename(columns = {"ENCOUNTER" : "Id", "TOTALCOST" : "TOTAL_MED_COST"}, inplace=True)
df_encounters_final = pd.merge(df_encounters_final, med_cost[["Id","TOTAL_MED_COST"]], on="Id", how= "left")
df_encounters_final["TOTAL_MED_COST"] = df_encounters_final["TOTAL_MED_COST"].fillna(df_encounters_final["TOTAL_MED_COST"].median())

#Alter bei Aufnahme erechnen in df_encounters_final
df_patients_age = df_patients.rename(columns={"Id" :"PATIENT"})
df_encounters_final = pd.merge(df_encounters_final, df_patients_age[["PATIENT", "BIRTHDATE"]], on="PATIENT", how="left")

df_encounters_final["START"] = pd.to_datetime(df_encounters_final["START"], errors="coerce")
df_encounters_final["BIRTHDATE"] = pd.to_datetime(df_encounters_final["BIRTHDATE"], errors="coerce")
df_encounters_final["AGE_AT_ENCOUNTER"] = (df_encounters_final["START"] - df_encounters_final["BIRTHDATE"]).dt.days/365
df_encounters_final["AGE_AT_ENCOUNTER"] = df_encounters_final["AGE_AT_ENCOUNTER"].astype(int)
df_encounters_final["START"] = df_encounters_final["START"].dt.strftime("%m/%d/%Y, %H:%M:%S")
df_encounters_final.drop(columns = "BIRTHDATE", inplace=True)

display(df_encounters_final)